# Goedel-Prover-V2 Google Colab Notebook

This notebook provides optimized execution for Goedel-Prover-V2 on Google Colab with limited GPU memory.

## Setup Environment

In [ ]:
# Clone repository
!git clone https://github.com/Goedel-LM/Goedel-Prover-V2.git
%cd Goedel-Prover-V2

In [ ]:
# Install dependencies
!pip install -q torch transformers accelerate bitsandbytes
!pip install -q vllm jsonlines tqdm pandas

In [ ]:
# Check GPU availability
import torch

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"GPU: {gpu_name}")
    print(f"Memory: {gpu_memory:.1f} GB")
else:
    print("No GPU available, will use CPU")

## Option 1: CPU Execution (Slow but Reliable)

In [ ]:
# Create CPU inference script
cpu_inference_code = '''#!/usr/bin/env python3
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import json
from tqdm import tqdm
import os

# Configuration
MODEL_PATH = "Goedel-LM/Goedel-Prover-V2-8B"
DATA_PATH = "dataset/test.jsonl"
OUTPUT_DIR = "results/cpu_test"
N_SAMPLES = 2
MAX_LENGTH = 2048

os.makedirs(OUTPUT_DIR, exist_ok=True)

print("Loading model on CPU...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype=torch.float32,
    trust_remote_code=True,
    device_map="cpu"
)
model.eval()

# Load problems
problems = []
with open(DATA_PATH, "r") as f:
    for line in f:
        problems.append(json.loads(line))

results = []
for problem in tqdm(problems[:5]):  # Process first 5 problems
    lean_code = problem.get("lean4_code", "")
    if not lean_code:
        continue
    
    prompt = f"""Complete the following Lean 4 proof:\n\n{lean_code}\n\n### Proof:"""
    
    for i in range(N_SAMPLES):
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=MAX_LENGTH)
        
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=MAX_LENGTH,
                temperature=0.7,
                do_sample=True,
                pad_token_id=tokenizer.pad_token_id
            )
        
        generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
        results.append({
            "problem_id": f"{problem.get('name')}_g{i}",
            "generated_proof": generated[len(prompt):]
        })

# Save results
with open(f"{OUTPUT_DIR}/results.json", "w") as f:
    json.dump(results, f, indent=2)

print(f"Generated {len(results)} proofs")
'''

with open('cpu_inference.py', 'w') as f:
    f.write(cpu_inference_code)

print("CPU inference script created")

In [ ]:
# Run CPU inference
!python cpu_inference.py

## Option 2: Quantized GPU Execution (Recommended)

In [ ]:
# Create quantized inference script
with open('colab_quantized_run.py', 'w') as f:
    f.write(open('colab_quantized_run.py').read())

print("Quantized inference script ready")

In [ ]:
# Run quantized inference
!python colab_quantized_run.py

## Option 3: Quick Test with 4-bit Quantization

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import json

# 4-bit quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model with 4-bit quantization
print("Loading 8B model with 4-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    "Goedel-LM/Goedel-Prover-V2-8B",
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(
    "Goedel-LM/Goedel-Prover-V2-8B",
    trust_remote_code=True
)

print("Model loaded successfully!")

In [ ]:
# Test with a simple problem
test_problem = """theorem test_theorem (n : Nat) : n + 0 = n := by"""

prompt = f"Complete the following Lean 4 proof:\n\n{test_problem}\n"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        temperature=0.7,
        do_sample=True
    )

generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated proof:")
print(generated[len(prompt):])

## Run Compilation and Summary

In [ ]:
# Compile generated proofs
!python src/compile.py \
    --input_path results/cpu_test/to_inference_codes.json \
    --output_path results/cpu_test/compilation.json \
    --cpu 2

In [ ]:
# Generate summary
!python src/summarize.py \
    --input_path results/cpu_test/compilation.json \
    --output_dir results/cpu_test/summary

## View Results

In [ ]:
# Display summary
import json
import pandas as pd

try:
    with open('results/cpu_test/summary/summary.json', 'r') as f:
        summary = json.load(f)
    print("Summary:")
    print(json.dumps(summary, indent=2))
except FileNotFoundError:
    print("Summary file not found. Run compilation and summarization first.")